In [1]:
!pip install nltk
!pip install gradio
!pip install tensorflow
!pip install Sastrawi
!pip install sentence-transformers
!pip install transformers
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.6/381.6 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.7 MB

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import nltk

nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
import gradio as gr
import time
import json
import numpy as np
import nltk
import random
import tensorflow as tf

from sentence_transformers import SentenceTransformer, util
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import WordNetLemmatizer
from transformers import AutoTokenizer
from pyngrok import conf, ngrok

MODEL_TF = '/content/drive/My Drive/Colab Notebooks/IDSS_LEARN/Byu_model_train/BOT_DEPLOY/model/model.h5' # load dari Drive bisa, dari files via bisa
MODEL_TORCH = '/content/drive/My Drive/Colab Notebooks/IDSS_LEARN/Byu_model_train/BOT_DEPLOY/model/model_transformer' # load dari Drive.
# khusus MODEL_TORCH, diusahakan foldernya yang akan diload, bukan yang lain.


def load_intent_data(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data


intent_data = load_intent_data('/content/drive/My Drive/Colab Notebooks/IDSS_LEARN/Byu_model_train/Datasets_copy.json') # load dari Drive bisa, dari files via bisa
stemmer = StemmerFactory().create_stemmer()
wordnet_lemmatizer = WordNetLemmatizer()
tokenizer = AutoTokenizer.from_pretrained("indolem/indobert-base-uncased")
model = SentenceTransformer(MODEL_TORCH)
modelTF = tf.keras.models.load_model(MODEL_TF)

def create_document(intents):
    all_words = []
    all_tags = []
    documents = []
    ignore = ['!', '.', '?', ',']

    for intent in intents['intents']:
        for pattern in intent['patterns']:
            w = tokenizer.encode(pattern, add_special_tokens=True)
            w = [tokenizer.decode(w_token) for w_token in w if tokenizer.decode(
                w_token) not in ignore]

            w = [stemmer.stem(word) for word in w]

            all_words.extend(w)
            documents.append((w, intent['tag']))
            if intent['tag'] not in all_tags:
                all_tags.append(intent['tag'])

    all_words = [w.lower() for w in all_words if w not in ignore]
    all_words = sorted(list(set(all_words)))
    all_tags = sorted(list(set(all_tags)))
    return all_words, all_tags, documents


total_words, total_tags, total_documents = create_document(intent_data)


def clean_up_userQuery(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    sentence_words = [wordnet_lemmatizer.lemmatize(
        word.lower()) for word in sentence_words]
    return sentence_words


def create_BoW(sentence, words, show_details=False):
    sentence_words = clean_up_userQuery(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details == True:
                    print("Word Found in the Bag : %s" % w)
    return (np.array(bag))


def match_intent(input_token, intent_data):
    input_embeddings = model.encode(input_token, convert_to_tensor=True)

    best_match = None
    best_similarity = -1

    for intent in intent_data['intents']:
        for pattern in intent['patterns']:
            pattern_embedding = model.encode(pattern, convert_to_tensor=True)
            similarity = util.pytorch_cos_sim(
                input_embeddings, pattern_embedding)[0].item()

            if similarity > best_similarity:
                best_similarity = similarity
                best_match = (intent, pattern, similarity)

    return best_match


context = {}
ERROR_TRESHOLD = 0.50
words = total_words
classes = total_tags

def send_matched_patterns(inp):
    inp = inp.lower()
    res = response(inp, show_details=False)
    cls = classify(inp)
    return res, cls


def classify(sentence):
    p = create_BoW(sentence, words, show_details=False)
    results = modelTF.predict(np.array([p]))[0]
    results = [[i, r] for i, r in enumerate(results) if r > ERROR_TRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list


def response(sentence, userID='NCI', show_details=False):
    results = classify(sentence)
    if results:
        while results:
            for i in intent_data['intents']:
                if i['tag'] == results[0][0]:
                    if 'context_set' in i:
                        if show_details:
                            print('context:', i['context_set'])
                        context[userID] = i['context_set']
                    if not 'context_filter' in i or \
                            (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details:
                            print('Tag:', i['tag'])
                        return random.choice(i['responses'])
            results.pop(0)


def chatbot(inp, history):
    for i in range(len(inp)):
        time.sleep(0.1)
        inp = inp.lower()
        if inp == "quit" or inp == "q" or inp == "bye":
            yield "Terima kasih sudah menggunakan chatbot ini"
        else:
            best_match = match_intent(inp, intent_data)
            if best_match is not None:
                matched_intent, matched_pattern, similarity = best_match
                if similarity * 100 > 80:
                    send_matched_patterns(matched_pattern)
                    yield "Macia : " + send_matched_patterns(matched_pattern)[0]

port = "7860" # diisikan port pada gradio, caranya? jalankan ini 1 kali, lalu lihat "https://localhost:786x", nah 786x inilah port pada gradio.
#jika sudah tau portnya, hentikan proses ini atau di interrupt, dan ketik port gradio yang barusan aktif, lalu jalankan kembali proses ini.

conf.get_default().auth_token = "2YEVZBCTOPdGPfsVym6eJzceRUp_P9QXiG8DmMvvpUj29VJd" # di dapat dari dashboard ngrok.
# bagaimana cara mendapatkan authtoken? daftar akun -> pergi ke section getting started -> di klik sampai segitiga sampai muncul 2 sub section
# lalu pilih "Your Authtoken" -> pastekan deh aututokennya.

public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"https://localhost:{}\"".format(public_url, port))

demo = gr.ChatInterface(chatbot, title="Chatbot Mental Health Mahasiswa").queue()

if __name__ == '__main__':
    demo.launch(share=False, debug=True)


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/234k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

 * ngrok tunnel "https://3a4e-104-154-210-118.ngrok-free.app" -> "https://localhost:7860"
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

1/1 [==============================] - 0s 19ms/step
Keyboard interruption in main thread... closing server.
